In [1]:
from google.colab import drive
import os
import sys
drive.mount('/content/drive')
path1="/content/drive/MyDrive/HollowKnight-main"
os.chdir(path1);
os.listdir(path1);

Mounted at /content/drive


In [ ]:
!pip uninstall keras
!pip install keras==2.4.3
!pip uninstall tensorflow
!pip install tensorflow==2.4.1
!pip uninstall tensorflow-gpu
!pip install tensorflow-gpu==2.4.1

In [1]:
import time
import numpy as np
import os
while episode < max_episode:
  episode += 1
  if episode % 5 == 0:
    model.load_target_model()
  while True:
    with open('/content/drive/Othercomputers/我的笔记本电脑/share/command.txt', 'r') as f:
      command=f.read()
    if command=='1':
      break
    time.sleep(0.1)
  while True:
    flag=os.path.exists('/content/drive/Othercomputers/我的笔记本电脑/share/input.npz') 
    if flag==True:
      data=np.load('/content/drive/Othercomputers/我的笔记本电脑/share/input.npz')
      break
    time.sleep(0.1)
  batch_station1=data['arr_0']
  batch_actions1=data['arr_1']
  batch_reward1=data['arr_2']
  batch_next_station1=data['arr_3']
  batch_done1=data['arr_4']
  batch_station2=data['arr_5']
  batch_actions2=data['arr_6']
  batch_reward2=data['arr_7']
  batch_next_station2=data['arr_8']
  batch_done2=data['arr_9']
  act_q, act_loss = algorithm.act_learn(batch_station2, batch_actions2, batch_reward2, batch_next_station2, batch_done2)
  move_q, move_loss = algorithm.move_learn(batch_station1, batch_actions1, batch_reward1, batch_next_station1, batch_done1)
  with open('/content/drive/Othercomputers/我的笔记本电脑/share/output.txt', 'w') as f:
    f.write(str(float(act_q))+' '+str(float(act_loss))+' '+str(float(move_q))+' '+str(float(move_loss)))
  model.save_mode()
  with open('/content/drive/Othercomputers/我的笔记本电脑/share/command.txt', 'w') as f:
    f.write('0')

NameError: ignored

In [4]:
episode=179
max_episode=30000
LEARNING_RATE = 0.00001  
GAMMA = 0.8
WIDTH = 400
HEIGHT = 200
ACTION_DIM = 7
FRAMEBUFFERSIZE = 4
INPUT_SHAPE = (FRAMEBUFFERSIZE, HEIGHT, WIDTH, 3)
model = Model(INPUT_SHAPE, ACTION_DIM)
#model.save_mode()
model.load_model()
model.load_target_model()
algorithm = DQN(model, gamma=GAMMA, learnging_rate=LEARNING_RATE)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4, 200, 400, 0                                            
__________________________________________________________________________________________________
conv1 (Conv3D)                  (None, 3, 99, 199, 3 1760        input_1[0][0]                    
__________________________________________________________________________________________________
conv2 (Conv3D)                  (None, 2, 97, 197, 4 27696       conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv3D)                  (None, 1, 95, 195, 6 55360       conv2[0][0]                      
______________________________________________________________________________________________

In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model as mod
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.layers import add, Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, \
    Activation, GlobalAveragePooling2D, Conv3D, MaxPooling3D, GlobalAveragePooling3D, Reshape, Lambda

import time
import os


class BasicBlock(layers.Layer):
    def __init__(self, filter_num, name, stride=1, **kwargs):
        super(BasicBlock, self).__init__(**kwargs)
        self.filter_num = filter_num
        self.stride = stride
        self.layers = []
        self.conv1 = layers.Conv2D(filter_num, 3, strides=stride, padding='same', name=name + '_1')
        # self.bn1=layers.BatchNormalization()
        self.relu = layers.Activation('relu')

        self.conv2 = layers.Conv2D(filter_num, 3, strides=1, padding='same', name=name + '_2')
        # self.bn2 = layers.BatchNormalization()
        self.layers.append(self.conv1)
        self.layers.append(self.conv2)
        # self.layers.append(self.bn1)
        # self.layers.append(self.bn2)
        if stride != 1:
            self.downsample = models.Sequential()
            self.downsample.add(layers.Conv2D(filter_num, 1, strides=stride))
            self.layers.append(self.downsample)
        else:
            self.downsample = lambda x: x

    def get_layer(self, index):
        return self.layers[index]

    def get_layers(self):
        return self.layers

    def call(self, input, training=None):
        out = self.conv1(input)
        # out=self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        # out=self.bn2(out)

        identity = self.downsample(input)
        output = layers.add([out, identity])  # ***
        output = tf.nn.relu(output)
        return output

    def get_config(self):
        config = {
            'filter_num':
                self.filter_num,
            'stride':
                self.stride
        }

        base_config = super(BasicBlock, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class Model:
    def __init__(self, input_shape, act_dim):
        self.act_dim = act_dim
        self.input_shape = input_shape
        self._build_model()
        self.act_loss = []
        self.move_loss = []

    def load_model(self):

        # self.shared_model = load_model("./model/shared_model.h5", custom_objects={'BasicBlock': BasicBlock})
        if os.path.exists("/content/drive/Othercomputers/我的笔记本电脑/share/act_part.h5"):
            print("load action model")
            self.act_model = models.Sequential()
            self.private_act_model = load_model("/content/drive/Othercomputers/我的笔记本电脑/share/act_part.h5", custom_objects={'BasicBlock': BasicBlock})
            # self.act_model.add(self.shared_model)
            self.act_model.add(self.private_act_model)

        if os.path.exists("/content/drive/Othercomputers/我的笔记本电脑/share/move_part.h5"):
            print("load move model")
            self.move_model = models.Sequential()
            self.private_move_model = load_model("/content/drive/Othercomputers/我的笔记本电脑/share/move_part.h5", custom_objects={'BasicBlock': BasicBlock})
            # self.move_model.add(self.shared_model)
            self.move_model.add(self.private_move_model)

    def load_target_model(self):
        if os.path.exists("/content/drive/Othercomputers/我的笔记本电脑/share/act_part.h5"):
            self.target_act_model = load_model("/content/drive/Othercomputers/我的笔记本电脑/share/act_part.h5", custom_objects={'BasicBlock': BasicBlock})
        if os.path.exists("/content/drive/Othercomputers/我的笔记本电脑/share/move_part.h5"):
            self.target_move_model = load_model("/content/drive/Othercomputers/我的笔记本电脑/share/move_part.h5", custom_objects={'BasicBlock': BasicBlock})

    def save_mode(self):
        print("save model")
        self.private_act_model.save("/content/drive/Othercomputers/我的笔记本电脑/share/act_part.h5")
        self.private_move_model.save("/content/drive/Othercomputers/我的笔记本电脑/share/move_part.h5")

    def build_resblock(self, filter_num, blocks, name="Resnet", stride=1):
        res_blocks = models.Sequential()
        # may down sample
        res_blocks.add(BasicBlock(filter_num, name + '_1', stride))
        # just down sample one time
        for pre in range(1, blocks):
            res_blocks.add(BasicBlock(filter_num, name + '_2', stride=1))
        return res_blocks

    # use two groups of net, one for action, one for move
    def _build_model(self):

        # ------------------ build evaluate_net ------------------

        # self.shared_model = models.Sequential()
        # self.private_act_model = models.Sequential()
        # self.private_move_model = models.Sequential()

        # shared part
        # pre-process block
        # self.shared_model.add(Conv2D(64, (2,3,3),strides=(1,2,2), input_shape=self.input_shape, name='conv1'))
        # # self.shared_model.add(BatchNormalization(name='b1'))
        # self.shared_model.add(Activation('relu'))
        # self.shared_model.add(MaxPooling3D(pool_size=(2,2,2), strides=1, padding="VALID", name='p1'))

        # # resnet blocks
        # self.shared_model.add(self.build_resblock(64, 2, name='Resnet_1'))
        # self.shared_model.add(self.build_resblock(80, 2, name='Resnet_2', stride=2))
        # self.shared_model.add(self.build_resblock(128, 2, name='Resnet_3', stride=2))

        # output layer for action model
        inputs = Input(shape=self.input_shape)
        x = Conv3D(32, (2, 3, 3), strides=(1, 2, 2), input_shape=self.input_shape, name='conv1', activation='relu')(
            inputs)
        x = Conv3D(48, (2, 3, 3), strides=(1, 1, 1), input_shape=self.input_shape, name='conv2', activation='relu')(x)
        x = Conv3D(64, (2, 3, 3), strides=(1, 1, 1), input_shape=self.input_shape, name='conv3', activation='relu')(x)
        x = Lambda(lambda x: tf.reduce_sum(x, 1))(x)
        x = self.build_resblock(64, 2, name='Resnet_1')(x)
        x = self.build_resblock(96, 2, name='Resnet_2', stride=2)(x)
        x = self.build_resblock(128, 2, name='Resnet_3', stride=2)(x)
        x = self.build_resblock(256, 2, name='Resnet_4', stride=2)(x)
        x = GlobalAveragePooling2D(name='pooling')(x)
        v_func = Dense(1, name='v_func')(x)
        a_func = Dense(self.act_dim, name='a_func')(x)
        a_func = add([a_func, -tf.reduce_mean(a_func, keepdims=True)])
        outputs = add([v_func, a_func])
        self.private_act_model = mod(inputs=inputs, outputs=outputs)

        # self.private_act_model.add(Reshape((1, -1)))
        # self.private_act_model.add(CuDNNLSTM(32))
        # self.private_act_model.add(Dense(self.act_dim, name="d1"))        # action model
        self.private_act_model.summary()
        self.act_model = models.Sequential()
        # self.act_model.add(self.shared_model)
        self.act_model.add(self.private_act_model)

        #target_act_model
        input = Input(shape=self.input_shape)
        y = Conv3D(32, (2, 3, 3), strides=(1, 2, 2), input_shape=self.input_shape, name='conv1', activation='relu')(
            input)
        y = Conv3D(48, (2, 3, 3), strides=(1, 1, 1), input_shape=self.input_shape, name='conv2', activation='relu')(y)
        y = Conv3D(64, (2, 3, 3), strides=(1, 1, 1), input_shape=self.input_shape, name='conv3', activation='relu')(y)
        y = Lambda(lambda x: tf.reduce_sum(x, 1))(y)
        y = self.build_resblock(64, 2, name='Resnet_1')(y)
        y = self.build_resblock(96, 2, name='Resnet_2', stride=2)(y)
        y = self.build_resblock(128, 2, name='Resnet_3', stride=2)(y)
        y = self.build_resblock(256, 2, name='Resnet_4', stride=2)(y)
        y = GlobalAveragePooling2D(name='pooling')(y)
        v = Dense(1, name='v_func')(y)
        a = Dense(self.act_dim, name='a_func')(y)
        a = add([a, -tf.reduce_mean(a, keepdims=True)])
        output = add([v, a])
        self.target_act_model = mod(inputs=input, outputs=output)

        # output layer for move model
        _inputs = Input(shape=self.input_shape)
        _x = Conv3D(32, (2, 3, 3), strides=(1, 2, 2), input_shape=self.input_shape, name='conv1', activation='relu')(
            _inputs)
        _x = Conv3D(48, (2, 3, 3), strides=(1, 1, 1), input_shape=self.input_shape, name='conv2', activation='relu')(_x)
        _x = Conv3D(64, (2, 3, 3), strides=(1, 1, 1), input_shape=self.input_shape, name='conv3', activation='relu')(_x)
        _x = Lambda(lambda x: tf.reduce_sum(x, 1))(_x)
        _x = self.build_resblock(64, 2, name='Resnet_1')(_x)
        _x = self.build_resblock(96, 2, name='Resnet_2', stride=2)(_x)
        _x = self.build_resblock(128, 2, name='Resnet_3', stride=2)(_x)
        _x = self.build_resblock(256, 2, name='Resnet_4', stride=2)(_x)
        _x = GlobalAveragePooling2D(name='pooling')(_x)
        _v_func = Dense(1, name='v_func')(_x)
        _a_func = Dense(self.act_dim, name='a_func')(_x)
        _a_func = add([_a_func, -tf.reduce_mean(_a_func, keepdims=True)])
        _outputs = add([_v_func, _a_func])
        self.private_move_model = mod(inputs=_inputs, outputs=_outputs)

        # movement model
        self.move_model = models.Sequential()
        # self.move_model.add(self.shared_model)
        self.move_model.add(self.private_move_model)

        # target_move_model
        _input = Input(shape=self.input_shape)
        _y = Conv3D(32, (2, 3, 3), strides=(1, 2, 2), input_shape=self.input_shape, name='conv1', activation='relu')(
            _input)
        _y = Conv3D(48, (2, 3, 3), strides=(1, 1, 1), input_shape=self.input_shape, name='conv2', activation='relu')(_y)
        _y = Conv3D(64, (2, 3, 3), strides=(1, 1, 1), input_shape=self.input_shape, name='conv3', activation='relu')(_y)
        _y = Lambda(lambda x: tf.reduce_sum(x, 1))(_y)
        _y = self.build_resblock(64, 2, name='Resnet_1')(_y)
        _y = self.build_resblock(96, 2, name='Resnet_2', stride=2)(_y)
        _y = self.build_resblock(128, 2, name='Resnet_3', stride=2)(_y)
        _y = self.build_resblock(256, 2, name='Resnet_4', stride=2)(_y)
        _y = GlobalAveragePooling2D(name='pooling')(_y)
        _v = Dense(1, name='v_func')(_y)
        _a = Dense(self.act_dim, name='a_func')(_y)
        _a = add([_a, -tf.reduce_mean(_a, keepdims=True)])
        _output = add([_v, _a])
        self.target_act_model = mod(inputs=_input, outputs=_output)

    #     # ------------------ build target_model ------------------
    #    # shared part

    #     self.shared_target_model = models.Sequential()
    #     # pre-process block
    #     self.shared_target_model.add(Conv3D(64, (2,3,3),strides=(1,2,2), input_shape=self.input_shape, name='conv1'))
    #     self.shared_target_model.add(BatchNormalization(name='b1'))
    #     self.shared_target_model.add(Activation('relu'))
    #     self.shared_target_model.add(MaxPooling3D(pool_size=(2,2,2), strides=1, padding="VALID", name='p1'))

    #     # resnet blocks
    #     self.shared_target_model.add(self.build_resblock(64, 2, name='Resnet_1'))
    #     self.shared_target_model.add(self.build_resblock(80, 2, name='Resnet_2', stride=2))
    #     self.shared_target_model.add(self.build_resblock(128, 2, name='Resnet_3', stride=2))

    #     # output layer for action model
    #     self.private_act_target_model = models.Sequential()
    #     self.private_act_target_model.add(self.build_resblock(200, 2, name='Resnet_4', stride=2))
    #     self.private_act_target_model.add(GlobalAveragePooling3D())
    #     # self.private_act_target_model.add(Reshape((1, -1)))
    #     # self.private_act_target_model.add(CuDNNLSTM(32))
    #     self.private_act_target_model.add(Dense(self.act_dim, name="d1", kernel_regularizer=regularizers.L2(0.001)))

    #     # action model
    #     self.act_target_model = models.Sequential()
    #     self.act_target_model.add(self.shared_target_model)
    #     self.act_target_model.add(self.private_act_target_model)

    #     # output layer for move model
    #     self.private_move_target_model = models.Sequential()
    #     self.private_move_target_model.add(self.build_resblock(200, 2, name='Resnet_4', stride=2))
    #     self.private_move_target_model.add(GlobalAveragePooling3D())
    #     # self.private_move_target_model.add(Reshape((1, -1)))
    #     # self.private_move_target_model.add(CuDNNLSTM(32))
    #     self.private_move_target_model.add(Dense(4, name="d1", kernel_regularizer=regularizers.L2(0.001)))

    #     # action model
    #     self.move_target_model = models.Sequential()
    #     self.move_target_model.add(self.shared_target_model)
    #     self.move_target_model.add(self.private_move_target_model)

    def predict(self, input):

        input = tf.expand_dims(input, axis=0)
        # shard_output = self.shared_model.predict(input)
        pred_move = self.private_move_model(input)
        pred_act = self.private_act_model(input)
        return pred_move, pred_act

import tensorflow as tf
import numpy as np


class DQN:
    def __init__(self, model, gamma=0.9, learnging_rate=0.0001):
        self.model = model
        self.act_dim = model.act_dim
        self.act_model = model.act_model
        self.move_model = model.move_model
        self.gamma = gamma
        self.lr = learnging_rate
        # --------------------------训练模型--------------------------- # 
        self.act_model.optimizer = tf.optimizers.Adam(learning_rate=self.lr)
        self.act_model.loss_func = tf.losses.MeanSquaredError()

        self.move_model.optimizer = tf.optimizers.Adam(learning_rate=self.lr)
        self.move_model.loss_func = tf.losses.MeanSquaredError()
        # self.act_model.train_loss = tf.metrics.Mean(name="train_loss")
        # ------------------------------------------------------------ #
        self.act_global_step = 0
        self.move_global_step = 0
        self.update_target_steps = 100  # 每隔200个training steps再把model的参数复制到target_model中

    # train functions for act model
    def act_predict(self, obs):
        """ 使用self.act_model的value网络来获取 [Q(s,a1),Q(s,a2),...]
        """
        return self.act_model.predict(obs)

    def act_train_step(self, action, features, next_features, labels):
        """ 训练步骤
        """
        with tf.GradientTape() as tape:
            # 计算 Q(s,a) 与 target_Q的均方差，得到loss
            predictions = self.act_model(features, training=True)
            next_pred = self.act_model(next_features, training=True)
            best_a = []
            for i in range(next_pred.shape[0]):
                best_a.append(np.argmax(next_pred[i, :]))
            target_q = []
            target_pred = self.model.target_act_model(next_features, training=False)
            for i in range(next_pred.shape[0]):
                target_q.append(target_pred[i, best_a[i]])
            target_q = tf.convert_to_tensor(target_q)
            enum_action = list(enumerate(action))
            pred_action_value = tf.gather_nd(predictions, indices=enum_action)
            loss = self.act_model.loss_func(labels + self.gamma * target_q, pred_action_value)
        gradients = tape.gradient(loss, self.act_model.trainable_variables)
        self.act_model.optimizer.apply_gradients(zip(gradients, self.act_model.trainable_variables))
        self.model.act_loss.append(loss)
        return np.mean(target_q), loss
        # self.act_model.train_loss.update_state(loss)

    def act_train_model(self, action, features, next_features, labels, epochs=1):
        """ 训练模型
        """
        q_sum = 0
        loss_sum = 0
        for epoch in tf.range(1, epochs + 1):
            q, loss = self.act_train_step(action, features, next_features, labels)
            q_sum = q_sum + q
            loss_sum = loss_sum + loss
        return q_sum/epochs, loss_sum/epochs
    def act_learn(self, obs, action, reward, next_obs, terminal):
        """ 使用DQN算法更新self.act_model的value网络
        """
        # print('learning')
        # 每隔200个training steps同步一次model和target_model的参数
        # if self.act_global_step % self.update_target_steps == 0:
        #     self.act_replace_target()

        # 从target_model中获取 max Q' 的值，用于计算target_Q

        q, loss = self.act_train_model(action, obs, next_obs, reward, epochs=1)
        self.act_global_step += 1
        return q, loss
        # print('finish')

    def act_replace_target(self):  # ???
        '''预测模型权重更新到target模型权重'''
        for i, l in enumerate(
                self.act_target_model.get_layer(index=1).get_layer(index=0).get_layer(index=0).get_layers()):
            l.set_weights(self.act_model.get_layer(index=1).get_layer(index=0).get_layer(index=0).get_layer(
                index=i).get_weights())
        for i, l in enumerate(
                self.act_target_model.get_layer(index=1).get_layer(index=0).get_layer(index=1).get_layers()):
            l.set_weights(self.act_model.get_layer(index=1).get_layer(index=0).get_layer(index=1).get_layer(
                index=i).get_weights())

        # for i, l in enumerate(self.act_target_model.get_layer(index=1).get_layer(index=1).get_layer(index=0).get_layers()):
        #     l.set_weights(self.act_model.get_layer(index=1).get_layer(index=1).get_layer(index=0).get_layer(index=i).get_weights())
        # for i, l in enumerate(self.act_target_model.get_layer(index=1).get_layer(index=1).get_layer(index=1).get_layers()):
        #     l.set_weights(self.act_model.get_layer(index=1).get_layer(index=1).get_layer(index=1).get_layer(index=i).get_weights())

        # for i, l in enumerate(self.act_target_model.get_layer(index=1).get_layer(index=2).get_layer(index=0).get_layers()):
        #     l.set_weights(self.act_model.get_layer(index=1).get_layer(index=2).get_layer(index=0).get_layer(index=i).get_weights())
        # for i, l in enumerate(self.act_target_model.get_layer(index=1).get_layer(index=2).get_layer(index=1).get_layers()):
        #     l.set_weights(self.act_model.get_layer(index=1).get_layer(index=2).get_layer(index=1).get_layer(index=i).get_weights())

        self.act_target_model.get_layer(index=1).get_layer(index=2).set_weights(
            self.act_model.get_layer(index=1).get_layer(index=2).get_weights())

        # self.act_target_model.get_layer(index=1).get_layer(index=6).set_weights(self.act_model.get_layer(index=1).get_layer(index=6).get_weights())

    # train functions for move_model

    def move_predict(self, obs):
        """ 使用self.move_model的value网络来获取 [Q(s,a1),Q(s,a2),...]
        """
        return self.move_model.predict(obs)

    def move_train_step(self, action, features, next_features, labels):
        """ 训练步骤
        """
        with tf.GradientTape() as tape:
            # 计算 Q(s,a) 与 target_Q的均方差，得到loss
            predictions = self.move_model(features, training=True)
            next_pred = self.move_model(next_features, training=True)
            best_a = []
            for i in range(next_pred.shape[0]):
                best_a.append(np.argmax(next_pred[i, :]))
            target_q = []
            target_pred = self.model.target_move_model(next_features, training=False)
            for i in range(next_pred.shape[0]):
                target_q.append(target_pred[i, best_a[i]])
            target_q = tf.convert_to_tensor(target_q)
            enum_action = list(enumerate(action))
            pred_action_value = tf.gather_nd(predictions, indices=enum_action)
            loss = self.move_model.loss_func(labels + self.gamma * target_q, pred_action_value)
        gradients = tape.gradient(loss, self.move_model.trainable_variables)
        self.move_model.optimizer.apply_gradients(zip(gradients, self.move_model.trainable_variables))
        self.model.move_loss.append(loss)
        return np.mean(target_q), loss
        # self.move_plot_loss()
        # print("Move loss: ", loss)
        # self.move_model.train_loss.update_state(loss)

    def move_train_model(self, action, features, next_features, labels, epochs=1):
        """ 训练模型
        """
        q_sum = 0
        loss_sum = 0
        for epoch in tf.range(1, epochs + 1):
            q, loss = self.move_train_step(action, features, next_features, labels)
            q_sum = q_sum + q
            loss_sum = loss_sum +loss
        return q_sum/epochs, loss_sum/epochs

    def move_learn(self, obs, action, reward, next_obs, terminal):
        """ 使用DQN算法更新self.move_model的value网络
        """
        q, loss = self.move_train_model(action, obs, next_obs, reward, epochs=1)
        self.move_global_step += 1
        return q, loss

    def move_replace_target(self):
        '''预测模型权重更新到target模型权重'''

        for i, l in enumerate(
                self.move_target_model.get_layer(index=1).get_layer(index=0).get_layer(index=0).get_layers()):
            l.set_weights(self.move_model.get_layer(index=1).get_layer(index=0).get_layer(index=0).get_layer(
                index=i).get_weights())
        for i, l in enumerate(
                self.move_target_model.get_layer(index=1).get_layer(index=0).get_layer(index=1).get_layers()):
            l.set_weights(self.move_model.get_layer(index=1).get_layer(index=0).get_layer(index=1).get_layer(
                index=i).get_weights())

        # for i, l in enumerate(self.move_target_model.get_layer(index=1).get_layer(index=1).get_layer(index=0).get_layers()):
        #     l.set_weights(self.move_model.get_layer(index=1).get_layer(index=1).get_layer(index=0).get_layer(index=i).get_weights())
        # for i, l in enumerate(self.move_target_model.get_layer(index=1).get_layer(index=1).get_layer(index=1).get_layers()):
        #     l.set_weights(self.move_model.get_layer(index=1).get_layer(index=1).get_layer(index=1).get_layer(index=i).get_weights())

        # for i, l in enumerate(self.move_target_model.get_layer(index=1).get_layer(index=2).get_layer(index=0).get_layers()):
        #     l.set_weights(self.move_model.get_layer(index=1).get_layer(index=2).get_layer(index=0).get_layer(index=i).get_weights())
        # for i, l in enumerate(self.move_target_model.get_layer(index=1).get_layer(index=2).get_layer(index=1).get_layers()):
        #     l.set_weights(self.move_model.get_layer(index=1).get_layer(index=2).get_layer(index=1).get_layer(index=i).get_weights())

        self.move_target_model.get_layer(index=1).get_layer(index=2).set_weights(
            self.move_model.get_layer(index=1).get_layer(index=2).get_weights())

        # self.move_target_model.get_layer(index=1).get_layer(index=6).set_weights(self.move_model.get_layer(index=1).get_layer(index=6).get_weights())

    def replace_target(self):
        # print("replace target")

        # copy conv3d_1
        self.model.shared_target_model.get_layer(index=0).set_weights(
            self.model.shared_model.get_layer(index=0).get_weights())
        # copy batchnormalization_1
        self.model.shared_target_model.get_layer(index=1).set_weights(
            self.model.shared_model.get_layer(index=1).get_weights())

        # copy shard_resnet block
        for i, l in enumerate(self.model.shared_target_model.get_layer(index=4).get_layer(index=0).get_layers()):
            l.set_weights(
                self.model.shared_model.get_layer(index=4).get_layer(index=0).get_layer(index=i).get_weights())
        for i, l in enumerate(self.model.shared_target_model.get_layer(index=4).get_layer(index=1).get_layers()):
            l.set_weights(
                self.model.shared_model.get_layer(index=4).get_layer(index=1).get_layer(index=i).get_weights())

        for i, l in enumerate(self.model.shared_target_model.get_layer(index=5).get_layer(index=0).get_layers()):
            l.set_weights(
                self.model.shared_model.get_layer(index=5).get_layer(index=0).get_layer(index=i).get_weights())
        for i, l in enumerate(self.model.shared_target_model.get_layer(index=5).get_layer(index=1).get_layers()):
            l.set_weights(
                self.model.shared_model.get_layer(index=5).get_layer(index=1).get_layer(index=i).get_weights())

        self.move_replace_target()
        self.act_replace_target()
